In [2]:
import os
import sys

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import s3fs
import seaborn as sns

sys.path.append("/Users/fgu/dev/projects/entropy")
import entropy.data.cleaners as cl
import entropy.data.creators as cr
import entropy.data.selectors as sl
import entropy.figures.figures as figs
import entropy.helpers.aws as ha
import entropy.helpers.data as hd
import entropy.helpers.helpers as hh
from entropy import config

sns.set_style("whitegrid")
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)
pd.set_option("max_colwidth", None)
%config InlineBackend.figure_format = 'retina'
%load_ext autoreload
%autoreload 2

fs = s3fs.S3FileSystem(profile="3di")

In [3]:
dfs, df = hd.read_samples(["777", "XX7"])
hd.inspect(df)

Time for read_sample    : 1.88 seconds
Time for read_sample    : 3.30 minutes
(7,531,116, 31)


,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,entropy_sptac
0,19510816,2014-03-07,107,9.20,paypal sandrpowell 9001,paypal,spend,services,0.0,gl3 4,2010-05-10,40k to 50k,1984.0,2014-05-11,238820,2020-08-16 12:13:00,barclaycard,credit card,2014-07-18,2017-09-12,True,NaN,paypal,enjoyment,home electronics,home electronics,u,201403,NaN,27530.926758,2.036569
1,19510817,2014-03-08,107,6.94,paypal smartbitsso 9001,paypal,spend,services,0.0,gl3 4,2010-05-10,40k to 50k,1984.0,2014-05-11,238820,2020-08-16 12:13:00,barclaycard,credit card,2014-07-18,2017-09-12,True,NaN,paypal,enjoyment,home electronics,home electronics,u,201403,NaN,27530.926758,2.036569


In [391]:
sl.income_amount(dfs).income.describe()

count    65964.000000
mean     25857.369215
std      11574.543785
min       6658.520020
25%      19063.750000
50%      24264.291016
75%      32108.140625
max      74680.907812
Name: income, dtype: float64

In [444]:
%%timeit


def max_accounts(df, max_accounts=5):
    """No more than 10 active accounts in any year"""
    year = pd.Grouper(freq="Y", key="date")
    usr_max = (
        df.groupby(["user_id", year]).account_id.nunique().groupby("user_id").max()
    )
    users = usr_max[usr_max <= max_accounts].index
    return df[df.user_id.isin(users)]


max_accounts(dfs)

35.3 ms ± 1.72 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [447]:
# %%timeit

def max_accounts(df, max_accounts=5):
    """No more than 10 active accounts in any year"""
    year = pd.Grouper(freq="Y", key="date")
    max_num_accounts_in_year = (
        df.groupby(["user_id", year])
        .account_id.transform("nunique")
        .groupby(df.user_id)
        .transform(max)
        # .values
    )
    return max_num_accounts_in_year
    return df.loc[max_num_accounts_in_year <= max_accounts]


max_accounts(dfs)

,id,date,user_id,amount,desc,merchant,tag_group,tag,user_female,user_postcode,user_registration_date,user_salary_range,user_yob,account_created,account_id,account_last_refreshed,account_provider,account_type,data_warehouse_date_created,data_warehouse_date_last_updated,debit,latest_balance,merchant_business_line,tag_auto,tag_manual,tag_up,updated_flag,ym,balance,income,entropy_sptac
7118,20201743,2014-02-24,58777,10.000000,lnk po woodgate 04 cd 2813 22feb 14,NaN,spend,other_spend,1.0,b32 3,2014-05-19,NaN,1979.0,2014-05-19,124626,2015-07-17 07:23:00,halifax personal banking,current,2014-07-18,2017-10-23,True,8.380000,personal,cash,NaN,cash,u,201402,3.750026,14802.468750,2.880838
7119,20201744,2014-02-24,58777,-5.000000,paypal 22 mdbremoved,paypal,spend,services,1.0,b32 3,2014-05-19,NaN,1979.0,2014-05-19,124626,2015-07-17 07:23:00,halifax personal banking,current,2014-07-18,2017-11-13,False,8.380000,paypal,enjoyment,NaN,paypal account,u,201402,3.750026,14802.468750,2.880838
7120,20201749,2014-02-25,58777,40.000000,lnk birmingham new cd 2813 25feb 14,NaN,spend,other_spend,1.0,b32 3,2014-05-19,NaN,1979.0,2014-05-19,124626,2015-07-17 07:23:00,halifax personal banking,current,2014-07-18,2017-10-23,True,8.380000,personal,cash,NaN,cash,u,201402,170.770035,14802.468750,2.880838
7121,20201698,2014-02-25,58777,190.000000,lnk po woodgate 04 cd 0412 25feb 14,NaN,spend,other_spend,1.0,b32 3,2014-05-19,NaN,1979.0,2014-05-19,124625,2015-07-17 07:23:00,halifax personal banking,current,2014-07-18,2017-10-23,True,0.990000,personal,cash,NaN,cash,u,201402,1.449991,14802.468750,2.880838
7122,20201748,2014-02-25,58777,-79.000000,mdbremoved 5664,NaN,transfers,transfers,1.0,b32 3,2014-05-19,NaN,1979.0,2014-05-19,124626,2015-07-17 07:23:00,halifax personal banking,current,2014-07-18,2017-08-13,False,8.380000,personal,transfers,NaN,NaN,u,201402,170.770035,14802.468750,2.880838
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
69416,802950445,2020-07-30,578777,5.000000,mdbremoved sto,NaN,NaN,NaN,1.0,bl8 2,2020-03-29,30k to 40k,1988.0,2020-03-30,1649664,2020-08-16 20:04:00,barclays,current,2020-07-31,1900-01-01,True,266.179993,NaN,NaN,NaN,NaN,c,202007,288.880005,23881.320871,2.557548
69417,802950444,2020-07-30,578777,3.450000,arthur lane on 29 jul clp,NaN,transfers,transfers,1.0,bl8 2,2020-03-29,30k to 40k,1988.0,2020-03-30,1649664,2020-08-16 20:04:00,barclays,current,2020-07-31,1900-01-01,True,266.179993,NaN,transfers,lunch or snacks,lunch or snacks,c,202007,288.880005,23881.320871,2.557548
69418,803166582,2020-07-31,578777,15.700000,wickes bury on 30 jul clp,wickes,spend,household,1.0,bl8 2,2020-03-29,30k to 40k,1988.0,2020-03-30,1649664,2020-08-16 20:04:00,barclays,current,2020-08-01,1900-01-01,True,266.179993,wickes,diy,NaN,diy,c,202007,266.180054,23881.320871,2.557548
69419,803166583,2020-07-31,578777,7.000000,pets at home ltd on 30 jul clp,pets at home,spend,retail,1.0,bl8 2,2020-03-29,30k to 40k,1988.0,2020-03-30,1649664,2020-08-16 20:04:00,barclays,current,2020-08-01,1900-01-01,True,266.179993,pets at home,pet - everyday or food,NaN,pet - everyday or food,c,202007,266.180054,23881.320871,2.557548


In [440]:
pd.testing.assert_frame_equal(max_accounts(dfs), max_accounts2(dfs))